In [1]:

# Install necessary libraries
!pip install -q openai langchain langchain-openai langchain-community openai-whisper sentence-transformers pdf2image
!apt-get install poppler-utils
!pip install --upgrade Pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 13.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 

In [2]:
# Import libraries
from langchain_openai import ChatOpenAI
from openai import OpenAI
from IPython.display import display, Markdown
from sentence_transformers import SentenceTransformer
import whisper
import pandas as pd
import base64
from pdf2image import convert_from_path
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import os
import torch

In [3]:
# Check if a CUDA GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Transcribe an audio file using Whisper
model = whisper.load_model("base", device=device)
input_file = "/content/starbucks-q3.mp3"
result = model.transcribe(input_file)
transcription_text = result['text']

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 63.6MiB/s]


In [5]:
# Check transcription output
print("Transcription Output:")
print(transcription_text)

Transcription Output:
 2024. And with that, I'll now tell Nicole over to Luxembourg. Thank you Tiffany and thank you for joining us this afternoon. Let me start by laying out our results for the squatter. Our Q3 total company revenue was $9.1 billion up 1% year over year and 6% over Q2. Our global comparable store sales declined 3% year over year driven by a negative 2% Comcro in North America and a negative 14% Comcro in China and partially offset by strong performance in Japan. Our global operating margins contracted by 70 basis points to 16.7% and overall earnings per share for the quarter was 93 cents. Our total company results were in line with guidance but international performance particularly in China was challenged. We are not satisfied with the results but our actions are making an impact. Leading business and operational indicators are trending in the right direction ahead of our financial results and our runway for improvement is long. We see green shoots in our US business

In [6]:
# Save the transcription to a text file
os.makedirs("transcript", exist_ok=True)
with open("transcript/transcription.txt", "w") as f:
    f.write(transcription_text)

In [7]:
# Load transcription text and split into chunks of 100 characters
with open('transcript/transcription.txt', 'r', encoding='utf-8') as file:
    text = file.read()

audio_chunks = [text[i:i+100] for i in range(0, len(text), 100)]

# Print chunks to verify splitting
print(f"\nNumber of Chunks: {len(audio_chunks)}")
print("First few chunks:", audio_chunks[:3])


Number of Chunks: 142
First few chunks: [" 2024. And with that, I'll now tell Nicole over to Luxembourg. Thank you Tiffany and thank you for j", 'oining us this afternoon. Let me start by laying out our results for the squatter. Our Q3 total comp', 'any revenue was $9.1 billion up 1% year over year and 6% over Q2. Our global comparable store sales ']


In [8]:
# Load Sentence Transformer model and embed the text chunks
model = SentenceTransformer('clip-ViT-B-32')
audio_embeddings = model.encode(audio_chunks)

# Check shape of embeddings
print(f"\nAudio Embeddings Shape: {audio_embeddings.shape}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.



Audio Embeddings Shape: (142, 512)


In [9]:
# Convert each page of a PDF into images
pdf_path = '/content/3Q24-Earnings-Release.pdf'
output_folder = 'images'
os.makedirs(output_folder, exist_ok=True)

In [10]:
images = convert_from_path(pdf_path)
image_paths = []

for i, image in enumerate(images):
    image_path = f'images/page_{i + 1}.png'
    image.save(image_path, 'PNG')
    image_paths.append(image_path)

print(f"PDF pages have been converted to images and saved in '{output_folder}'")
print(f"\nNumber of Images: {len(image_paths)}")
print(f"Image Paths: {image_paths[:3]}")  # Print first 3 image paths to check

PDF pages have been converted to images and saved in 'images'

Number of Images: 17
Image Paths: ['images/page_1.png', 'images/page_2.png', 'images/page_3.png']


In [11]:
# Load the CLIP model for embedding images
image_model = SentenceTransformer('clip-ViT-B-32')
image_embeddings = []

for filename in os.listdir(output_folder):
    if filename.endswith('.png'):
        image_path = os.path.join(output_folder, filename)
        image = Image.open(image_path)
        embedding = image_model.encode(image)
        image_embeddings.append(embedding)

In [12]:
# Check image embeddings
print(f"\nNumber of Image Embeddings: {len(image_embeddings)}")
print(f"First Image Embedding Shape: {image_embeddings[0].shape}")


Number of Image Embeddings: 17
First Image Embedding Shape: (512,)


In [13]:
# Define a query and embed it
query = "How is the company doing financially?"
query_embeddings = model.encode(query)

In [14]:
# Compute similarity with transcription embeddings
audio_similarities = cosine_similarity([query_embeddings], audio_embeddings)[0]
top_k_audio_indices = audio_similarities.argsort()[-20:][::-1]

In [15]:
# Print top 5 similar audio chunks
print("\nTop 5 Most Similar Audio Chunks (Indices):", top_k_audio_indices[:5])
print("Top 5 Audio Similarities:", audio_similarities[top_k_audio_indices[:5]])


Top 5 Most Similar Audio Chunks (Indices): [ 93  94 107  13  83]
Top 5 Audio Similarities: [0.91604316 0.9142631  0.9071276  0.9053204  0.90503275]


In [16]:
# Compute cosine similarity between the query embedding and each image embedding
image_similarities = [(idx, cosine_similarity(query_embeddings.reshape(1, -1), embed.reshape(1, -1))[0][0])
                      for idx, embed in enumerate(image_embeddings)]


In [17]:
# Sort by similarity and get top 2 most similar images
top_2_images = sorted(image_similarities, key=lambda x: x[1], reverse=True)[:2]

In [18]:
# Combine the top transcription chunks into a single text context
text_context = ' '.join([audio_chunks[idx] for idx in top_k_audio_indices])

In [19]:
# Print combined text context
print(f"\nText Context (First 300 characters): {text_context[:300]}...")


Text Context (First 300 characters): ontinued growth of the program because the average active member spends materially more annually and  drives a high- lifetime value for the business than a non-member. Research also tells us that the m  50 percentage points. This combined with in-app offers helped drive a 10% year-over-year growth i...


In [20]:
# Convert the top 2 images to base64 format
base64frames = []
for idx, _ in top_2_images:
    image_path = image_paths[idx]
    with open(image_path, "rb") as img_file:
        base64frames.append(base64.b64encode(img_file.read()).decode('utf-8'))

In [21]:
# Print base64 encoding of first image
print(f"\nBase64 of First Image (First 100 characters): {base64frames[0][:100]}...")


Base64 of First Image (First 100 characters): iVBORw0KGgoAAAANSUhEUgAABqQAAAiYCAIAAAA+NVHkAAEAAElEQVR4nOzdeVxO6f84/qtNtNGmVErJRLJlL4ks2TO2KGtD9t3Y...


In [22]:
# Import Google Generative AI library
import google.generativeai as genai
from PIL import Image

# Get the API key from Google Colab
from google.colab import userdata
api_key = userdata.get('Gemini_API')

# Configure the Gemini API
genai.configure(api_key=api_key)

# Prepare image data for Gemini API using PIL.Image
image_objects = []
for idx, _ in top_2_images:
    image_path = image_paths[idx]
    # Open images with PIL
    img = Image.open(image_path)
    image_objects.append(img)

# Try with one of the available vision models from your list


In [23]:
try:
    # Using gemini-pro-vision which is in your available models list
    model = genai.GenerativeModel('gemini-pro-vision')

    # Create the prompt with text context from audio
    prompt = f"You are a financial adviser. Analyze the following financial information and respond to this query: '{query}'\n\nText from earnings call transcript: {text_context}"

    # Prepare content for the API request
    contents = [prompt]
    for img in image_objects:
        contents.append(img)

    # Generate response
    response = model.generate_content(contents)

    # Print response
    print("\n===== GEMINI API RESPONSE =====")
    print(response.text)

    # Save to file
    with open("gemini_analysis.txt", "w") as f:
        f.write(response.text)

    # Display the first part of the saved response to verify
    print("\nResponse saved to file. First 200 characters:")
    with open("gemini_analysis.txt", "r") as f:
        saved_content = f.read()
        print(saved_content[:200] + "...")

except Exception as e:
    print(f"\nError with vision model: {e}")

    # Fallback to one of the available text models from your list
    try:
        # Try gemini-1.5-pro which is in your available models list
        text_model = genai.GenerativeModel('gemini-1.5-pro')
        text_response = text_model.generate_content(
            f"You are a financial adviser. Analyze the following financial information and respond to this query: '{query}'\n\n"
            f"Text from earnings call transcript: {text_context}\n\n"
            f"Note: There are also images of financial statements that couldn't be processed."
        )

        print("\n===== TEXT-ONLY RESPONSE =====")
        # print(text_response.text)
        display(Markdown(text_response.text))

        with open("gemini_analysis.txt", "w") as f:
            f.write(text_response.text)

    except Exception as e2:
        print(f"\nText-only model also failed: {e2}")

        # Try one more model (an older version)
        try:
            basic_model = genai.GenerativeModel('models/text-bison-001')
            basic_response = basic_model.generate_text(
                f"You are a financial adviser. Analyze the following financial information and respond to this query: '{query}'\n\n"
                f"Text from earnings call transcript: {text_context}\n\n"
                f"Note: There are also images of financial statements that couldn't be processed."
            )

            print("\n===== BASIC TEXT MODEL RESPONSE =====")
            # print(basic_response.text)
            display(Markdown(basic_response.text))


            with open("gemini_analysis.txt", "w") as f:
                f.write(basic_response.text)

        except Exception as e3:
            print(f"\nAll models failed. Final error: {e3}")


Error with vision model: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro-vision:generateContent?%24alt=json%3Benum-encoding%3Dint: Gemini 1.0 Pro Vision has been deprecated on July 12, 2024. Consider switching to different model, for example gemini-1.5-flash.

===== TEXT-ONLY RESPONSE =====


Based on the earnings call transcript alone, the company's financial health appears mixed.  There are positive signs, but also significant challenges and uncertainties.

**Positive Indicators:**

* **Growth in active membership program:**  Increased spending by members and higher lifetime value suggests a successful loyalty program driving revenue.
* **Year-over-year growth:**  A 10% YoY growth, although the specific metric isn't mentioned, is a positive sign.
* **Focus on innovation:**  Investing in new products and relevant marketing indicates a proactive approach to maintaining competitiveness.
* **Productivity improvements:**  Plans to enhance productivity and unlock capital with short payback periods suggest efficiency initiatives.
* **Long-term vision:**  Focusing on customer engagement and value creation demonstrates a strategic approach to sustainable growth.
* **Initial success of a plan:** The mention of a plan working in its early stages is a positive, although more details are needed.

**Negative Indicators/Areas of Concern:**

* **International business weakness:**  Challenges in international markets pose a risk to overall performance.
* **Mass segment price war:**  Competing in a price war can negatively impact margins and profitability. This suggests a challenging competitive landscape.
* **Vague language:** The transcript lacks concrete financial figures. Terms like "materially more," "sizable opportunity," and "true step change improvement" are optimistic but don't provide quantifiable data.  The 10% YoY growth figure lacks context (revenue, profit, etc.?).
* **Omitted financial statements:**  The inability to process the financial statements hinders a comprehensive analysis. These statements would provide crucial information on revenue, profitability, cash flow, and debt levels, which are essential for assessing financial health.


**Overall Assessment:**

While the company highlights positive developments like membership growth and productivity initiatives, the transcript presents an incomplete picture. The absence of concrete financial data and the acknowledgment of international weakness and price wars raise concerns.  It's impossible to definitively assess the company's financial health without access to the financial statements.  I would need to review those statements to provide a more accurate and complete analysis.
